* 使用 zero-shot 和 (random) few-shot prompting
* 有標準答案的評估
* 沒有標準答案的評估

## 整合 GPT API 並透過 prompt-engineer 實作書籍分類

In [1]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [2]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [3]:
import requests
import json
from pprint import pp

In [4]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, format_type="text"):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens, "response_format": { "type": format_type } }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [15]:
import pandas as pd

df = pd.read_csv('./books-dataset-200.csv')

# 隨機挑 20 筆資料
df = df.sample(20).reset_index(drop=True)

### 無 CoT 進行書籍分類

In [17]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": """
          請根據以下書籍資訊，選擇最適合的技能分類。
          只需要回傳完整分類名稱，不要回傳其他文字，分類只能在以下選一個: 
          3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,
          Computer-Science,Computer-architecture,Data Science,Data-visualization
          ,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,
          Information-management,Java,Linux,Machine Learning,Operating-system,Python,
          R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,
          區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,
          機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,
          職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,
          軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits
          """      
        },
        {
            "role": "user",
            "content": f"書名: {row['title']} \n 描述: {row['description']}"
        }
    ]
    response = get_completion(messages, model="gpt-3.5-turbo")

    df.loc[index,"predict"] = response
    # df.at[index, 'predict'] = response
    # pp(response)

df

,title,description,category,tags,predict
0,Experimentation for Engineers: From A/B Testin...,Learn practical and modern experimental method...,機率統計學 Probability-and-statistics,機率,Data Science
1,Trigonometry for Dummies,Make trigonometry as easy as 1-2-3\r\n\r\nBeli...,高普特考,"研究所考試,高普特考",數學
2,Procedural 3D Modeling Using Geometry Nodes in...,An easy-to-follow illustrated guide to learnin...,3D建模 3D-modeling,Blender,3D 列印
3,網絡存儲技術,存儲技術是最近幾年IT行業最熱門的技術之一。本書全面深入地介紹了網絡存儲技術的基本原理，體系...,Information-management,資訊管理,資料庫
4,RF System Design of Transceivers for Wireless ...,This book is for RF Engineers and in particula...,Wireless-networks,無線網路,電路學 Electric-circuits
5,This Is Marketing: You Can't Be Seen Until You...,#1 Wall Street Journal Bestseller Instant New ...,行銷/網路行銷 Marketing,網路行銷,行銷/網路行銷
6,SQL Query Design Patterns and Best Practices: ...,Enhance your SQL query writing skills to provi...,SQL,"SQL,設計模式",資料庫
7,區域性銀行數字化轉型：方法論與實踐,本書分為10章。\r\n第1章系統介紹國內銀行業數字化轉型歷程，幫助讀者了解數字化轉型在國內...,企業資源規劃 Erp,"企業資源規劃,大數據運算,管理學",管理與領導 Management-leadership
8,Closing the Care Gap with Wearable Devices: In...,Patient-focused healthcare driven by COVID-19 ...,穿戴式裝置 Wearable,穿戴式裝置,資訊管理
9,數據中台：賦能企業實時經營與商業創新,隨著數字化的浪潮席捲而來，無論是個人還是企業，相信都已經真切地感受到了數字化的威力，而如何在...,企業資源規劃 Erp,"企業資源規劃,大數據運算,數據分析",資料庫


### 有 CoT 進行書籍分類 Few-shot prompting

In [54]:
examples = ''

for row in df.iterrows():
    examples = examples + f"""
    書名: {row[1]["title"]}
    描述: ```{row[1]["description"]}```
    分類: {row[1]["category"]}
    ----

"""

for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "請根據以下書籍資訊，選擇最適合的技能分類。只需要回傳完整分類名稱，不要回傳其他文字，分類只能在以下選一個: 3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,Computer-Science,Computer-architecture,Data Science,Data-visualization,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,Information-management,Java,Linux,Machine Learning,Operating-system,Python,R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits"
        },
        {
          "role": "user",
          "content": examples
        },
        {
          "role": "user",
          "content": f"書名: {row['title']} \n 描述: {row['description']}\n 分類:"
        }
    ]
    response = get_completion(messages, model="gpt-3.5-turbo")
    df.at[index, 'predict2'] = response


## 整合 GPT API 並透過 prompt-engineer 撰寫假書評推薦

In [21]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "依據書籍標題想像你是該領域的專家，並根據書籍標題和摘要，用親民的口語，撰寫推薦序?"
          
        },
        {
            "role": "user",
            "content": row["title"] + " " + row["description"]
        }
    ]
    
    response = get_completion(messages)
    df.loc[index,"recommend"] = response



In [22]:
df[["title", "recommend"]]

,title,recommend
0,Experimentation for Engineers: From A/B Testin...,嘿，各位工程師朋友們！想要在科技和交易領域中掌握實用且現代的實驗方法嗎？那這本《Experi...
1,Trigonometry for Dummies,"Howdy, math enthusiasts and learners! Are you ..."
2,Procedural 3D Modeling Using Geometry Nodes in...,"Hey there, fellow 3D modeling enthusiasts! If ..."
3,網絡存儲技術,嘿，各位科技迷們！如果你對網絡存儲技術感興趣，那這本《網絡存儲技術》絕對是你的不二之選！這本...
4,RF System Design of Transceivers for Wireless ...,嘿！你對無線通訊的 RF 系統設計感興趣嗎？如果是的話，這本《RF System Desig...
5,This Is Marketing: You Can't Be Seen Until You...,嘿！你想知道如何在市場上脫穎而出嗎？《This Is Marketing》這本書絕對能幫到你...
6,SQL Query Design Patterns and Best Practices: ...,嘿！你想要提升 SQL 查詢寫作技能嗎？如果是的話，這本《SQL Query Design ...
7,區域性銀行數字化轉型：方法論與實踐,嘿！你想知道如何讓區域性銀行跟上數位化的步伐嗎？這本書就是你的好幫手！它一共有10章，從介紹...
8,Closing the Care Gap with Wearable Devices: In...,嘿，大家好！我是健康科技領域的專家，今天要向大家推薦一本書，《Closing the Car...
9,數據中台：賦能企業實時經營與商業創新,嘿，各位企業領袖和業務人員！你們有感受到數字化的力量了嗎？想要在這個數字化時代中脫穎而出，就...



## LLM model 評估手法 (回歸本質: AI模型: 訓練 -> 驗證 -> 測試)
Evaluation 可分為三種情況:

* 有標準答案，可以算準確率 (一般不太準)
* 有參考答案，可以比較兩者相似度: 讓 LLM 幫你評分有多相似 (較符合常理)
* 沒有參考答案，只有評分標準: 讓 LLM 幫你打分數 (增強式學習法, 模型自己玩自己??)

### 準確率評估，和資料集的標注分類 category 相比

In [56]:
# 計算兩個欄位一致的筆數
correct_predictions = (df['category'] == df['predict']).sum()

# 計算準確率
accuracy = correct_predictions / len(df)


print(f"zero shot learning 準確率: {accuracy:.2%}")


# 計算兩個欄位一致的筆數
correct_predictions2 = (df['category'] == df['predict2']).sum()

# 計算準確率
accuracy2 = correct_predictions2 / len(df)


print(f"few shot learning 準確率: {accuracy2:.2%}")



zero shot learning 準確率: 5.00%
few shot learning 準確率: 15.00%


* 模型可換成用 gpt-4-turbo-preview 看看，準確率可以大幅上升。只是成本高很多.... XD
* 之後課程還會教一招 dynamic few-shot prompting 技巧，聰明的找出相似的範例
* 之後課程還會教如何用 function calling 來做這題，也可以提高準確率

## 評估產生的推薦

prompt 參考出處: https://spellbook.scale.com/

注意: 評分最好也設計 CoT，

設定規格:
範例 => 標準 => 背景描述(系統設定)

In [57]:
criteria = """
答案必須 
1. 需用台灣繁體中文(除了書名可用英文) 
2.足夠吸引人瀏覽 
3.和書籍內容相關 
4.順口能被記住"
"""


for index, row in df.iterrows():
    question = "根據以下書籍資料，如果我想推薦給別人，應該要說什麼? \n" + row["title"] + " \n" + row["description"]
    answer = row["recommend"]

    prompt = f"""
    [Question]

    {question}

    [The Start of Assistant's Answer]

    {answer}

    [The End of Assistant's Answer]

    [Criteria]

    {criteria}

    [System]

    We would like to request your feedback on the performance of an AI assistant in response to the user question displayed above.
    Please rate their response according to the criteria defined above. Do not consider any other criteria.
    The assistant receives an overall score on a scale of 1 to 5, where a higher score indicates better adherence to the criteria.
    Please provide a explanation of your evaluation, avoiding any potential bias, using taiwanese chinese.
    Then output only one value indicating the scores for the assistant.

    [Output JSON example]
    {{
      "reasoning": "a explanation of your evaluation>"
      "score": "integer"
    }}
    """

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    response = get_completion(messages, model="gpt-4-1106-preview", max_tokens=1000, format_type="json_object") # 評估需要推理得用 gpt-4 效果才好
    eval_data = json.loads(response)

    df.at[index, 'recommend_score'] = eval_data["score"]
    df.at[index, 'recommend_reason'] = eval_data["reasoning"]



C:\Users\user\AppData\Local\Temp\ipykernel_22516\2591313563.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'recommend_score'] = eval_data["score"]


In [61]:
df[["title", "recommend", "recommend_score", "recommend_reason"]]
df[["title", "recommend", "recommend_reason"]]


for i in df.iterrows():
    print(f"""
    書名: {i[1]["title"]}
    推薦序: {i[1]["recommend"]}
    推薦序評分: {i[1]["recommend_score"]}
    推薦序評分原因: {i[1]["recommend_reason"]}
    ----
    """)


    書名: Experimentation for Engineers: From A/B Testing to Bayesian Optimization
    推薦序: 嘿，各位工程師朋友們！想要在科技和交易領域中掌握實用且現代的實驗方法嗎？那這本《Experimentation for Engineers: From A/B Testing to Bayesian Optimization》絕對是你的好幫手！

這本書不僅教你如何進行A/B測試，還會帶你探索更高階的方法，幫助你優化機器學習系統、量化交易策略等等。無論你是在金融、社交媒體等競爭激烈的行業中工作，這些實驗技巧都能讓你事半功倍。

快來學習如何利用實驗方法提升你的工程能力，成為行業中的佼佼者吧！
    推薦序評分: 5.0
    推薦序評分原因: 根據所提供的評分標準，助理的回答使用了台灣繁體中文，並且保留了書名的英文原文，符合第一點要求。回答的內容與書籍主題相關，提到了A/B測試和貝葉斯優化等實驗方法，並且指出這些技巧在金融和社交媒體等行業的應用，滿足第三點要求。此外，回答的語氣活潑，用詞吸引人，能夠引起目標讀者的興趣，符合第二點要求。最後，回答的結尾「快來學習如何利用實驗方法提升你的工程能力，成為行業中的佼佼者吧！」是一句簡潔有力，容易記住的句子，達到了第四點要求。
    ----
    

    書名: Trigonometry for Dummies
    推薦序: Howdy, math enthusiasts and learners! Are you ready to conquer trigonometry like a pro? Well, look no further than "Trigonometry for Dummies"! This book is your ultimate guide to mastering those sines, cosines, and tangents without breaking a sweat.

Let's face it, trigonometry can be intimidating, but fear not! With this book by your side, you'll breeze th